In [1]:
import sys
assert sys.version_info >= (3, 5)

In [2]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [3]:
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request

In [4]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

Get the CSV data in to a panda dataframe, might need to alter filepaths depending where this is saved

In [6]:
def load_data():
    return pd.read_csv("../data/winemag-data-130k-v2.csv")

wine = load_data()
wine.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Ideas for pre-processing:

Create Vintage feature by scraping data from title field

Maybe scrape description for descriptive keywords?

Maybe try to find the longitude and latitude of the winery for some scattergraph data


Remove taster_twitter_handle

Remove Unnamed id feature

Drop the unneeded Id and taster twitter handle columns

In [7]:
wine = wine.drop(['Unnamed: 0', 'taster_twitter_handle'], axis=1)

Create a new feature for the vintage of the wine, get the data from the title field

Use RegEx to find the vintage year, make sure year is between 1975 and 2022 to avoid incorrect data. Some names don't contain vintage year, might need to remove those

In [8]:
import re

In [9]:
def getVintage(wine):
    f = re.findall(r'\d\d\d\d', wine["title"])
    if len(f) == 0:
        return int()
    elif len(f) > 1:
        for y in f:
            if int(y) < 1975 and int(y) > 2022:
                del f[y]
    f = f[0]
    return f

In [10]:
wine["vintage"] = wine.apply(getVintage, axis=1)

Drop all records that have no indication of location

In [11]:
locCols = ["country", "province", "region_1"]
wine = wine.dropna(axis=0,subset=locCols, how='all', inplace=True)

This is a script that tries to get the longitude and latitude of each wineries location, it queries an online google maps esque webpage and gradually widens the search area until there is a match. It is not viable to get all of the data this way.

In [12]:
%pip install requests

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python310\\Scripts\\normalizer.exe' -> 'c:\\Python310\\Scripts\\normalizer.exe.deleteme'

You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


In [14]:
import requests
import urllib.parse

ModuleNotFoundError: No module named 'requests'

In [ ]:
def getCoords(wine):
    address = str(wine["winery"]) + "," + str(wine["province"]) + "," + str(wine["region_1"]) + "," + str(wine["country"])
    repCount = 0
    while True:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
        response = requests.get(url).json()
        if response:
            print(response[0]["lat"])
            print(response[0]["lon"])
            return [response[0]["lat"], response[0]["lon"]]
        address = address.split(',')
        address = ','.join(address[1:])

If we let the code run, we should create some code to populate the longitude and latitude of the other records of each winery. At this current point, only the first record of each winery has their long and lat updated

In [ ]:
# Get the first record of each unique winery
uniqueWineries = wine.groupby("winery", as_index=False).first()

uniqueWineries["longitude"] = uniqueWineries.apply(getCoords, axis=1)[0]
uniqueWineries["latitude"] = uniqueWineries.apply(getCoords, axis=1)[1]